In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import problem
import warnings
import glob
import re
pd.set_option('display.max_columns', None)
warnings.filterwarnings("ignore")

In [2]:
external_data = pd.read_csv('submissions/submit/external_data.csv', 
                            index_col=0)

In [3]:
external_data[["CloudCover","Events"]].isnull().sum()

CloudCover       0
Events        6745
dtype: int64

In [4]:
def fix_events_column(data_weather):
    events_cloud = data_weather[["CloudCover","Events"]].fillna(value="other")
    
    sun_index = ((events_cloud['Events'] == "other") 
    & (events_cloud['CloudCover'] <= 4))
    
    cloudy_index = ((events_cloud['Events'] == "other") 
    & (events_cloud['CloudCover'] > 4))
    
    data_weather["Events"][cloudy_index]= "Cloud"
    data_weather["Events"][sun_index]= "Sun"
    
    events = data_weather["Events"]
    extrem_weather_index = ((events == "Rain-Thunderstorm-Tornado") ^ 
    (events == "Fog-Rain-Snow-Thunderstorm") ^ 
    (events == "Rain-Snow-Thunderstorm") ^ \
    (events == "Fog-Rain-Hail-Thunderstorm") ^ 
    (events == "Rain-Hail-Thunderstorm")^ 
    (events == "Fog-Rain-Thunderstorm")^ 
    (events == "Rain-Thunderstorm")^ 
    (events == "Thunderstorm"))
    data_weather["Events"][extrem_weather_index] = "Extrem"
    
    snow_index = ((events == "Rain-Snow") ^ 
    (events == "Fog-Snow") ^ 
    (events == "Fog-Rain-Snow")^ 
    (events == "Snow"))
    
    data_weather["Events"][snow_index]= "Snow"
    
    fog_index = ((events == "Fog") 
    ^ (events == "Fog-Rain"))
    
    data_weather["Events"][fog_index]= "Fog"
    
    return data_weather

In [5]:
fix_events_column(external_data)["Events"].value_counts()

Sun       4756
Rain      2171
Cloud     1989
Extrem     812
Fog        739
Snow       573
Name: Events, dtype: int64

In [6]:
def fix_gust_speed(data_weather):
    gust_speed = data_weather["Max Gust SpeedKm/h"]
    gust_speed = gust_speed.fillna(value="other")
    index = (gust_speed == "other")
    data_weather["Max Gust SpeedKm/h"][index] = data_weather["Max Wind SpeedKm/h"][index].astype(float)
    
    return data_weather

In [7]:
external_data = fix_events_column(external_data)

In [8]:
external_data = fix_gust_speed(external_data)

In [9]:
external_data = external_data.drop(columns="Precipitationmm")

In [11]:
external_data.to_csv('submissions/submit/external_data.csv')